In [6]:
import os

MAX_BATCHES = 113

def checkpoint_time(checkpoint: str):
    checkpoint = checkpoint.split('-')
    epoch = int(checkpoint[0][5:])
    batch = int(checkpoint[1][5:-3])
    return epoch, batch

def make_schedule(n_batches):
    n = int((n_batches + 1) / 2)
    ret = [0, n]
    n = int((n + 1) / 2)
    i = 1
    end = False
    while True:
        prev = ret[-i:]
        for p in prev:
            if p-n not in ret and p-n > 0 : ret.append(p - n)
            if p+n not in ret and p+n < n_batches: ret.append(p + n)

        if end or n == 0: break
        end = n == 1
        n = int((n+1) / 2)
        i = i * 2
    return list(ret)

def normalize_batch(epoch, batch):
    return epoch + (batch / MAX_BATCHES)

def get_checkpoints(run_name: str):
    weights_dir = os.path.join('runs', 'detect', run_name, 'weights')
    checkpoints = sorted(
        [c for c in os.listdir(weights_dir) if 'batch' in c],
        key=lambda x: normalize_batch(*checkpoint_time(x))
    )
    return weights_dir, checkpoints

In [7]:
from ultralytics import YOLO
from custom_eval import eval_video, get_test_files, BINARY_KEYS
import json
TEST_DIR = "datasets/04_Test_Datasets/"

test_files = get_test_files(TEST_DIR)

def eval_checkpoints(run_name, frame_conf, class_conf, max_epochs=None):
    weights_dir, checkpoints = get_checkpoints(run_name)
    n_epochs = len(set([checkpoint_time(c)[0] for c in checkpoints]))
    n_batches = len(set([checkpoint_time(c)[1] for c in checkpoints]))
    checkpoints_2d = [
        [c for c in checkpoints if checkpoint_time(c)[0] == e]
        for e in range(n_epochs)
    ]
    schedule = make_schedule(n_batches)
    print(schedule)
    for batch in schedule:
        for epoch in range(min(n_epochs, max_epochs)):
            checkpoint = checkpoints_2d[epoch][batch]
            print(f"EVAL ON EPOCH {epoch}, BATCH {batch} ({checkpoint})")
            model = YOLO(os.path.join(weights_dir, checkpoint))
            res = eval_video(model, test_files, frame_conf, class_conf, BINARY_KEYS)
            real_batch = checkpoint_time(checkpoint)[1]
            res['epoch'] = epoch
            res['batch'] = real_batch
            with open(os.path.join('runs', 'detect', run_name, 'bias.jsonl'), 'a') as f:
                f.write(json.dumps(res) + "\n")



In [8]:
eval_checkpoints('binary_ft_checkpoints', frame_conf=1e-2, class_conf=0.5, max_epochs=5)

[0, 7, 3, 11, 1, 5, 9, 10, 12, 2, 4, 6, 8]
EVAL ON EPOCH 0, BATCH 0 (epoch0-batch0.pt)
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


/home/coinse/Documents/lm/spring2024/CS570-Video-Classification/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,
/home/coinse/Documents/lm/spring2024/CS570-Video-Classification/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 81/81 [03:01<00:00,  2.24s/it]


NUM CORRECT: 22 / 81 (0.27)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Neg': 10,
          'False_Pos': 6,
          'True_Neg': 63,
          'True_Pos': 2,
          'guess_rate': 0.09876543209876543},
 'bird': {'False_Neg': 6,
          'False_Pos': 47,
          'True_Neg': 26,
          'True_Pos': 2,
          'guess_rate': 0.6049382716049383},
 'plane': {'False_Neg': 43,
           'False_Pos': 6,
           'True_Neg': 14,
           'True_Pos': 18,
           'guess_rate': 0.2962962962962963}}
EVAL ON EPOCH 1, BATCH 0 (epoch1-batch0.pt)
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


 86%|████████▋ | 70/81 [02:26<00:23,  2.10s/it]


KeyboardInterrupt: 

In [ ]:
eval_checkpoints('full_ft_checkpoints', frame_conf=1e-2, class_conf=0.5, max_epochs=5)


NameError: name 'eval_checkpoints' is not defined